# Data Preparation

- Is everything prepared for modeling?
- What result does analyzing the data return?
- Is reduction of the data possible?

In [3]:
##
# IMPORTS
#
import os
import urllib.request
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
#import datetime as dt

##
# SETTINGS
#
# Render all plots inline (instead of popup)
%matplotlib inline

##
# RUN
#
# run previous notebook
%run ./01-data-understanding.ipynb


The dataset contains 70643 rows and 59 columns.


The columns contain data about: 

Index(['iso_code', 'continent', 'location', 'date', 'total_cases', 'new_cases',
       'new_cases_smoothed', 'total_deaths', 'new_deaths',
       'new_deaths_smoothed', 'total_cases_per_million',
       'new_cases_per_million', 'new_cases_smoothed_per_million',
       'total_deaths_per_million', 'new_deaths_per_million',
       'new_deaths_smoothed_per_million', 'reproduction_rate', 'icu_patients',
       'icu_patients_per_million', 'hosp_patients',
       'hosp_patients_per_million', 'weekly_icu_admissions',
       'weekly_icu_admissions_per_million', 'weekly_hosp_admissions',
       'weekly_hosp_admissions_per_million', 'new_tests', 'total_tests',
       'total_tests_per_thousand', 'new_tests_per_thousand',
       'new_tests_smoothed', 'new_tests_smoothed_per_thousand',
       'positive_rate', 'tests_per_case', 'tests_units', 'total_vaccinations',
       'people_vaccinated', 'people_fully_vaccinated',

## Data Restructuring

In [10]:
# New columns with date split into year, month and weekday (can beused as index for plotting)
df_src['year'] = df_src['date'].dt.year
df_src['month'] = df_src['date'].dt.month
df_src['weekday'] = df_src['date'].dt.weekday

# create subsets by topic
df_gdp = df_src[['continent', 'iso_code', 'location', 'year', 'month', 'weekday', 'date', 'gdp_per_capita', 'extreme_poverty', 'life_expectancy', 'human_development_index']]

df_covid = df_src[['continent', 'iso_code', 'location', 'year', 'month', 'weekday', 'date', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_cases_per_million', 'new_cases_per_million', 'total_deaths_per_million', 'new_deaths_per_million', 'reproduction_rate', 'stringency_index']]

df_vaccination = df_src[['continent', 'iso_code', 'location', 'year', 'month', 'weekday', 'date','total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'new_vaccinations', 'total_vaccinations_per_hundred', 'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred']]

df_tests = df_src[['continent', 'iso_code', 'location', 'year', 'month', 'weekday', 'date', 'new_tests', 'total_tests', 'total_tests_per_thousand', 'new_tests_per_thousand', 'positive_rate', 'tests_per_case', 'tests_units']]

df_population = df_src[['continent', 'iso_code', 'location', 'year', 'month', 'weekday', 'date', 'population', 'population_density', 'median_age', 'aged_65_older', 'aged_70_older']]

,continent,iso_code,location,year,month,weekday,date,population,population_density,median_age,aged_65_older,aged_70_older
0,Asia,AFG,Afghanistan,2020,2,0,2020-02-24,38928341.0,54.422,18.6,2.581,1.337


In [ ]:
# create subsets by location
def split_by_location(df):
    # copy df in df_tmp
    df_tmp = df.copy
    # while df_tmp is_not_empty
    while not df_tmp.empty:
        # take location at 0
        loc = df_tmp.head(1).location
        # filter by location into new df_sub
        df_sub = df_tmp[[df_tmp.location == loc]]
        # drop df_tmp where == location
        # TODO
        df_tmp.drop
    # return df_sub